In [5]:
import numpy as np          #Berechnung von Matrizen
import random               #Generierung zufälliger Zahlen oder anderer zufälliger Operationen
import matplotlib as plt    #Erstellung von Diagrammen
#import torch                #PyTorch Bibliothek 
#import torch.nn as nn
#import torchvision          #Verarbeitung von Bildern
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import SGD

In [6]:
# Parameter definieren
image_folder = "train/"  # Ordner mit den Bildern
image_size = (128, 128)  # Zielgröße der Bilder

# Listen für Bilder und Labels initialisieren
images = []
labels = []

# ImageDataGenerator für Datenaugmentation definieren
datagen = ImageDataGenerator(
    rotation_range=40,         # Zufällige Rotation im Bereich von 0 bis 40 Grad
    width_shift_range=0.2,     # Zufällige horizontale Verschiebung
    height_shift_range=0.2,    # Zufällige vertikale Verschiebung
    shear_range=0.2,           # Zufällige Scherung
    zoom_range=0.2,            # Zufälliger Zoom
    horizontal_flip=True,      # Zufälliges Spiegeln der Bilder
    fill_mode='nearest'        # Auffüllen von Pixeln nach Transformationen
)

# Durch den Ordner iterieren und Bilder laden
for filename in os.listdir(image_folder):
    # Pfad zum Bild erstellen
    img_path = os.path.join(image_folder, filename)

    # Bild laden und auf die Zielgröße skalieren
    img = load_img(img_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0  # Normalisieren auf [0, 1]

    # Augmentierung durchführen
    img_array = np.expand_dims(img_array, axis=0)  # Eine Batch-Dimension hinzufügen
    augmented_image = datagen.random_transform(img_array[0])  # Augmentiertes Bild

    # Augmentiertes Bild hinzufügen
    images.append(augmented_image)

    # Label bestimmen (dog = 1, cat = 0)
    if "dog" in filename.lower():
        labels.append(1)
    elif "cat" in filename.lower():
        labels.append(0)

# In NumPy-Arrays umwandeln
images = np.array(images)
labels = np.array(labels)

# Daten in Trainings- und Validierungsdaten aufteilen
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Ergebnisse ausgeben
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)


x_train shape: (20000, 128, 128, 3)
y_train shape: (20000,)
x_test shape: (5000, 128, 128, 3)
y_test shape: (5000,)


In [7]:
# CNN-Modell
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(128, 128, 3)),                     # Form der Eingabedaten: 128x128 Pixel, 3 Farbkanäle
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),          # 2D-Faltung zur Extraktion von Merkmalen; 32 Filter mit einer Größe von 3x3; ReLU-Aktivierungsfunktion
    tf.keras.layers.MaxPooling2D((2, 2)),                           # Dimensionsreduktion; Pooling-Größe von 2x2
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),          
    tf.keras.layers.MaxPooling2D((2, 2)), 
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),         
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),                                      # Umwandlung der 2D-Ausgabe in ein 1D-Array
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid'),                  # 2 Ausgabeneuronen; Softmax Aktivierungsfunktion 
])

# Modell kompilieren
model.compile(optimizer= 'adam',  # Optimierte Lernrate
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Trainingsstopp, sobald der Validierungsgenauigkeit beginnt zu sinken
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True)


# Training starten
EPOCHS = 20                              
history = model.fit(
    x_train, 
    y_train,  
    epochs=EPOCHS,   
    validation_data=(x_test, y_test),
    callbacks=[early_stopping],
)

Epoch 1/20
625/625 [==============================] - 127s 201ms/step - loss: 0.6777 - accuracy: 0.5629 - val_loss: 0.6440 - val_accuracy: 0.6392
Epoch 2/20
625/625 [==============================] - 128s 205ms/step - loss: 0.6149 - accuracy: 0.6579 - val_loss: 0.5852 - val_accuracy: 0.6946
Epoch 3/20
625/625 [==============================] - 127s 203ms/step - loss: 0.5749 - accuracy: 0.6991 - val_loss: 0.5455 - val_accuracy: 0.7294
Epoch 4/20
625/625 [==============================] - 126s 202ms/step - loss: 0.5480 - accuracy: 0.7222 - val_loss: 0.5541 - val_accuracy: 0.7254
Epoch 5/20
625/625 [==============================] - 128s 205ms/step - loss: 0.5203 - accuracy: 0.7408 - val_loss: 0.5352 - val_accuracy: 0.7304
Epoch 6/20
625/625 [==============================] - 137s 218ms/step - loss: 0.4957 - accuracy: 0.7592 - val_loss: 0.4866 - val_accuracy: 0.7616
Epoch 7/20
625/625 [==============================] - 129s 207ms/step - loss: 0.4679 - accuracy: 0.7782 - val_loss: 0.4689 -